<a href="https://colab.research.google.com/github/gschivley/FERC_714/blob/master/FERC_IPM_through_861.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fuzzywuzzy

In [ ]:
import re
import zipfile
import urllib
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from fuzzywuzzy import fuzz, process
import fuzzywuzzy

pd.set_option("max_rows", 50)

cwd = Path.cwd()

In [ ]:
# Download the FERC 714 data to a temp folder that google is nice enough to host
# No need to run if already downloaded.
url = 'https://www.ferc.gov/docs-filing/forms/form-714/data/form714-database.zip'
save_folder = cwd / "FERC"
save_folder.mkdir(parents=True, exist_ok=True)
ferc_data_path = save_folder / "form714-database"
if not ferc_data_path.exists():
    urllib.request.urlretrieve(url, save_folder / 'form714-database.zip')

    ### Unzip it
    with zipfile.ZipFile(save_folder / 'form714-database.zip', 'r') as zfile:
        zfile.extractall(ferc_data_path)

In [ ]:
# Also download/extract the 2012 EIA 861 data.
url = 'https://www.eia.gov/electricity/data/eia861/archive/zip/f8612012.zip'
save_folder = cwd / "EIA861"
save_folder.mkdir(parents=True, exist_ok=True)
eia861_data_path = save_folder / "f8612012"
if not eia861_data_path.exists():
    urllib.request.urlretrieve(url, save_folder / 'f8612012.zip')
    ### Unzip it
    with zipfile.ZipFile(save_folder / 'f8612012.zip', 'r') as zfile:
        zfile.extractall(eia861_data_path)

In [ ]:
# Some functions to find the best name match using tfidf and cosine similarity
def ngrams(string, n=3):
    string = re.sub(r'[,-./]|\sBD',r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]


def vectorize_tfidf(series1, series2):
    vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
    
    vectorizer.fit(pd.concat([series1, series2]))
    tf_idf_matrix_1 = vectorizer.transform(series1)
    tf_idf_matrix_2 = vectorizer.transform(series2)
    
    return tf_idf_matrix_1, tf_idf_matrix_2

def top_name_match(ferc_df, eia_df):
    ferc_series = ferc_df['respondent_name'].reset_index(drop=True)
    eia_series = eia_df['eia_name'].reset_index(drop=True)
    
    tfidf1, tfidf2 = vectorize_tfidf(ferc_series, eia_series)
    cos_sim = cosine_similarity(tfidf1, tfidf2)
    
    cols = [
        'respondent_name', 'eia_name', 'score',
    ]
    results_df = pd.DataFrame(columns=cols, index=ferc_series.index)
    results_df['respondent_name'] = ferc_series
    matched_entities = []
    for idx, name in ferc_series.iteritems():
        best_match = np.argmax(cos_sim[idx])
        score = cos_sim[idx].max()
        eia_name = eia_series[best_match]
        matched_entities.append(eia_name)
        results_df.loc[idx, 'eia_name'] = eia_name
        results_df.loc[idx, 'score'] = score
        
    results_df = results_df.merge(
        ferc_df.loc[:, ['respondent_name', "respondent_id", "eia_code"]],
        on='respondent_name', how='left'
    )
    
    results_df = results_df.merge(
        eia_df,
        on='eia_name', how='left'
    )
    
    return results_df

In [ ]:
# Create a list of respondents that report data in 2012

ferc_714 = pd.read_csv(
    ferc_data_path / "Part 3 Schedule 2 - Planning Area Hourly Demand.csv",
    parse_dates=["plan_date"], infer_datetime_format=True
)

valid_respondents = ferc_714.loc[ferc_714.report_yr == 2012, "respondent_id"].unique()
len(sorted(valid_respondents))

## Load the FERC respondents
This includes the FERC respondent ID and associated EIA code. These codes sometimes match utilities and sometimes match BAs.

In [ ]:
# Codes that I'm changing after manual investigation. Worth double checking.
alt_ferc_eia_codes = {
    272: 25470, # WAPA upper great plains east
    125: 2775, # CAISO
}

In [ ]:
ferc_respondents = pd.read_csv(
    ferc_data_path / "Respondent IDs.csv"
)
ferc_respondents.loc[:, "respondent_name"] = ferc_respondents.loc[:, "respondent_name"].str.strip()
ferc_respondents = ferc_respondents.loc[ferc_respondents["respondent_id"].isin(valid_respondents), :]

for ferc_id, eia_code in alt_ferc_eia_codes.items():
    ferc_respondents.loc[
        ferc_respondents["respondent_id"] == ferc_id, 
        "eia_code"
    ] = eia_code
    
# ferc_respondents_eia_map = ferc_respondents.set_index("respondent_id")
# ferc_respondents_eia_map = ferc_respondents_eia_map.drop(columns="respondent_name")
ferc_respondents.head()

## First match utilities
Following the method described in the SI of [this PNAS paper (Auffhammer et al, 2017)](https://www.pnas.org/content/114/8/1886), first match FERC respondents to EIA utilities. I'm using 2012 EIA-861 data to match with the 2012 load data. Start with matching against the service territory data since we eventually need to be matching counties anyway.

CAISO matches with the City of Albany (before the manual change I made above in `alt_ferc_eia_codes`) but all other utilities seem to match fine against respondents. A total of 71 respondents match with utilities.

In [ ]:
utility_data = pd.read_excel(eia861_data_path / "utility_data_2012.xls", skiprows=1)
utility_data.head()

In [ ]:
service_territory = pd.read_excel(eia861_data_path / "service_territory_2012.xls")
service_territory.head()

In [ ]:
st_utilities = service_territory[["Utility Number", "Utility Name"]].drop_duplicates()

In [ ]:
respondent_utility = pd.merge(
    ferc_respondents, 
    utility_data[["Utility Number", "Utility Name"]], 
    left_on="eia_code", 
    right_on="Utility Number", 
    how="left"
)
respondent_utility.loc[
    respondent_utility["Utility Number"].isin(st_utilities["Utility Number"]),
    "geo_data_available"
] = True

In [ ]:
respondent_utility

In [ ]:
respondent_utility.notna().all(axis=1).sum()

In [ ]:
respondent_utility_sa = pd.merge(
    ferc_respondents, 
    st_utilities, 
    left_on="eia_code", 
    right_on="Utility Number", 
    how="left"
)

In [ ]:
respondent_utility_sa

In [ ]:
respondent_utility_sa.notna().all(axis=1).sum()

In [ ]:
# Use fuzzywuzzy to get a quick score on the name matches
def name_match_score(row, eia_col):
    ferc_name = row["respondent_name"]
    utility_name = row[eia_col]
    
    if pd.isna(utility_name):
        return np.nan
    else:
        score = fuzz.partial_token_sort_ratio(ferc_name, utility_name)
        return score
    
respondent_utility_sa["name_score"] = respondent_utility_sa.apply(
    lambda row: name_match_score(row, "Utility Name"), axis=1
)

In [ ]:
respondent_utility["name_score"] = respondent_utility.apply(
    lambda row: name_match_score(row, "Utility Name"), axis=1
)
respondent_utility.sort_values("name_score")

In [ ]:
respondent_utility_sa.sort_values("name_score")

## Match remaining respondents against BAs.

In [ ]:
utility_ba = pd.read_excel(eia861_data_path / "balancing_authority_2012.xls")

eia_bas = utility_ba.loc[:, ["BA Code", "Balancing Authority Name"]].drop_duplicates()

In [ ]:
non_utility_respondents = respondent_utility_sa.loc[
    respondent_utility_sa.isnull().any(axis=1), 
    ["respondent_id", "respondent_name", "eia_code"]
]
respondent_ba = pd.merge(non_utility_respondents, eia_bas, left_on="eia_code", right_on="BA Code", how="left")

In [ ]:
non_utility_respondents = respondent_utility.loc[
    respondent_utility.isnull().any(axis=1), 
    ["respondent_id", "respondent_name", "eia_code"]
]
respondent_ba = pd.merge(non_utility_respondents, eia_bas, left_on="eia_code", right_on="BA Code", how="left")

In [ ]:
respondent_ba["name_score"] = respondent_ba.apply(
    lambda row: name_match_score(row, "Balancing Authority Name"), axis=1
)

In [ ]:
respondent_ba.sort_values("name_score").head(10)

In [ ]:
respondent_ba.sort_values("name_score").tail(30)

## String match remaining respondents

In [ ]:
ba_codes = utility_ba.loc[:, ["BA Code", "Balancing Authority Name"]].drop_duplicates()
ba_codes

In [ ]:
remaining_respondents = respondent_ba.loc[
    respondent_ba.isna().any(axis=1),
    ["respondent_id", "respondent_name", "eia_code"]
]

In [ ]:
# Create a dictionary of potential respondents with <eia code/number>: <name>
# from all the potential input files.
potential_resp_list = [
    st_utilities.set_index("Utility Number").to_dict()["Utility Name"],
    eia_bas.set_index("BA Code").to_dict()["Balancing Authority Name"],
    utility_ba[["Utility Number", "Utility Name"]].drop_duplicates().set_index("Utility Number").to_dict()["Utility Name"]
]
potential_resp_dict = {k: v for d in potential_resp_list for k, v in d.items()}

In [ ]:
# Find the best name match, then check to see if the EIA code also matches.
# The fact that some codes match means that I didn't catch all utilities/BAs
# in the sections above.

name_match = top_name_match(remaining_respondents, pd.DataFrame(potential_resp_dict.items(), columns=["eia_id", "eia_name"]))
name_match["eia_code_match"] = False
name_match.loc[
    name_match["eia_code"] == name_match["eia_id"],
    "eia_code_match"
] = True

In [ ]:
name_match.sort_values("score")

In [ ]:
name_match.query("eia_code_match==False")

In [ ]:
# This is my preliminary list of string matches to keep
use_name_match = [
    "Westar Energy (KPL)",
    "PacifiCorp - Part II Sch 2 (East & West combined)",
    "City of West Memphis"
]

# Keep matches where the codes are the same or the names are in the list above.
good_name_matches = name_match.loc[
    (name_match["eia_code_match"] == True)
    | (name_match["respondent_name"].isin(use_name_match)),
    :
].set_index("respondent_id")
good_name_matches

In [ ]:
manual_ipm_match = {
    128: "S_VACA", # Central Electric Power Cooperative, Inc. - located in SC
#     292: "S_SOU", # Southern Power Company
}

## Combine results and find all county matches

TO DO